In [37]:
import json
import random
from statistics import mean
import numpy as np
import sys
import os
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.compose import TransformedTargetRegressor
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
import matplotlib.pyplot as plt
from itertools import accumulate
import pickle
from pathlib import Path
import datetime

from tqdm import tqdm

In [38]:
def filterEventProb(m, mLeft, byLeft):
    #See if this event is possible, given the current state
    # m is the TurnEvent
    # mLeft is the player's number of cards left
    # byLeft is the number of bystander cards left

    # print(f'**FILTER EVENT PROB for mine={mLeft}, by={byLeft}')
    # m.printEvent()

    #It isn't possible if m.mine is greater than mLeft
    if m.mine > mLeft:
        # print('  0 -> too many of mine')
        return 0.0
    
    #It isn't possible if m.mine equals mLeft and m.bad is not None
    if m.mine == mLeft and m.bad:
        # print('  0 -> all mine + bad')
        return 0.0

    #It isn't possible if m.bad is 'by' and byLeft is 0
    if m.bad == 'by' and byLeft == 0:
        # print('  0 -> no by left')
        return 0.0
    
    #We don't have to check if it is theirs or assassin, because if 
    #there weren't at least 1 of those cards, the game would already be over

    # print('  ', m.p, ' -> unfiltered')
    return m.p

In [39]:
def getEventProbs(pM, mLeft, byLeft):
    #pM is the player's TurnEvent list
    #mLeft is the player's number of cards left
    #byLeft is the number of bystander cards left
    # This function will compute and return a new distribution 
    # over the TurnEvents in pM, given the context, 
    # i.e. restricting ourselves to only possible turnEvents

    x = list(range(36))
    x2 = [i+36 for i in x]
    p_prior = [m.p for m in pM]
    


    #Filter probs by possibility
    mProbsUN = np.asarray([filterEventProb(m, mLeft, byLeft) for m in pM])
    #Renormalize and return
    mProbs = mProbsUN / mProbsUN.sum()
    if np.isnan(mProbs).any():
        print('ERROR, THIS IS WHAT WE HAVE HERE!!!!')
        print('Mine left: ', mLeft)
        print('By Left: ', byLeft)
        for i, m in enumerate(pM):
            m.printEvent()
            print(' => Post prob : ', mProbs[i])

    # if mLeft < 3:
    #     plt.bar(x,p_prior,color='r',label='prior')
    #     plt.bar(x2,list(mProbs),color='b',label='post')
    #     plt.legend()
    #     plt.title(f'Probs for my left : {mLeft} by left : {byLeft}')
    #     plt.show()
    return mProbs

In [40]:
def simulateGame(pA, pB, aLeft, bLeft, byLeft):
    # Complete a game from this state, where 
    # pA and pB are the player's TurnEvent lists (distributions over outcomes for single turn)
    # aLeft is the number of cards A has left
    # bLeft is the number of cards B has left
    # byLeft is the number of bystander cards left
    #Return True if A wins this game simulation

    # Create the probabilities to draw from
    AProbs = [a.p for a in pA]
    BProbs = [b.p for b in pB]
    
    #Whose turn is it?  Assume it is A's (can easily pass this in to be more general)
    ATurn = True

    # print('Simulating Game')

    #Sit in this loop until the game ends and something is returned
    while True:
        # print(f'[a:{aLeft} | by:{byLeft} | b:{bLeft}]: A Turn? {ATurn}')
        if ATurn:
            #What does A do this turn?
            #Draw their turn result
            AProbs = getEventProbs(pA, aLeft, byLeft)
            turnResult = np.random.choice(pA, p=AProbs)
            # turnResult.printEvent()  #Display it out
            aLeft -= turnResult.mine  #Adjust state by how many of their cards they got
            # This happens first, because turn ends if bad thing happens
            if aLeft <= 0:
                # print(' A won normally')
                return True, 0 #A won
            if turnResult.bad:
                #Something bad happened, adjust state accordingly
                if turnResult.bad == 'theirs':
                    bLeft -= 1
                if turnResult.bad == 'by':
                    byLeft -= 1
                if turnResult.bad == 'assassin':
                    # print(' B won by assassin')
                    return False, 1

            #If A lost by turning over other things, return False
            #If all the bystanders are turned over, that doesn't matter
            if bLeft == 0:
                # print('  B won by A choosing b')
                return False, 0

        else:
            #Same as above, but flipped for player B (trues become falses, and vice versa)
            #What does B do this turn?
            BProbs = getEventProbs(pB, bLeft, byLeft)
            turnResult = np.random.choice(pB, p=BProbs)
            # for i, t, in enumerate(pB):
            #     print(f'***{i}***')
            #     t.printEvent()
            #     print(f'   p={BProbs[i]}')
            # print('+++SELECTED++++')
            # turnResult.printEvent()
            bLeft -= turnResult.mine
            if bLeft <= 0:
                # print(' B won normally')
                return False, 0 
            if turnResult.bad:
                if turnResult.bad == 'theirs':
                    aLeft -= 1
                if turnResult.bad == 'by':
                    byLeft -= 1
                if turnResult.bad == 'assassin':
                    # print(' A won by assassin')
                    return True, 1

            if aLeft == 0:
                # print('  A won by B choosing a')
                return True, 0

        #Switch whose turn it is
        ATurn = not ATurn
    

In [41]:

def calculateWinPercentage(pA, pB, aLeft, bLeft, byLeft, N):
    #Estimate the win percentage for player A, where the inputs are:
    #  - pA is the list of A's TurnEvents
    #  - pB is the list of B's TurnEvents
    #  - aLeft is the number of cards A has left
    #  - bLeft is the number of cards B has left
    #  - byLeft is the number of bystander cards left

    assassinWins = 0
    Awins = 0
    for i in range(N):
        Awin, assassinWin = simulateGame(pA, pB, aLeft, bLeft, byLeft)
        assassinWins += assassinWin
        if Awin:
            Awins += 1

    # print(f'{assassinWins} assassin wins out of {N} games.')
    return Awins/N, assassinWins

In [42]:
class TurnEvent():
    def __init__(self, mine, bad, P):
        self.mine = mine
        self.bad = bad
        self.p = P

    def printEvent(self):
        print('[TurnEvent] (P =', round(self.p,4), ') Mine: ', self.mine, 'Bad: ', self.bad)

In [43]:
def create_string(arr):
    res = ''
    for num in arr:
        res += str(num)
    return res

In [44]:
#Generates all the possible events
def generate_pos_events():
    res = []
    for r in range(10):
        c = 0
        for b in [None, 'theirs', 'by', 'assassin']:
            
            if r == 0 and (b is None):
                c += 1
                continue
            if r == 9 and (b is not None):
                c += 1
                continue

            key = [r, 0, 0, 0]
            if c > 0:
                key[c] = 1
            c += 1
            res.append(create_string(key))
    return res
pos_events = generate_pos_events()

In [45]:
def create_events(pos_events,probs):
    #We pass in pos_events because order matters
    P = []

    for event, p in zip(pos_events,probs):
        r = int(event[0])
        if int(event[1]) == 1:
            b = "theirs"
        elif int(event[2]) == 1:
            b = "by"
        elif int(event[3]) == 1:
            b = "assassin"
        else:
            b = None

        newEvent = TurnEvent(r, b, p)
        P.append(newEvent)
    return P

In [46]:
def create_vector(AP, BP):
    return np.array(AP) - np.array(BP)

In [47]:
#This function will create a random distribution over outcomes
#Hand tuned to give reasonable vectors
def createRandomProbVector(n):
    #Default high value
    # S = 7
    S = np.random.randint(low=1,high=9)

    #scaler for high value for good things
    # m = 4
    m = np.random.randint(low=1, high=5)

    #scaler for high value for getting one right
    # w = 9
    w = np.random.randint(low=2,high=12)

    #high value for all the assassins
    # A = 2
    A = np.random.randint(low=1, high=8)
    h = S*np.ones(n)
    #Default high for all stats is 2, besides those listed
    h[0] = m*S   #1 Blue
    h[1] = m*S   #1 Bystander
    h[3] = w*S   #1 Correct
    h[4] = m*S   #1 Correct & 1 Blue
    h[5] = m*S   #1 Correct & 1 Bystander
    h[7] = m*S   #2 Correct
    h[8] = m*S   #2 Correct & 1 Blue
    h[9] = m*S   #2 Correct & 1 Bystander
    h[11] = m*S   #3 Correct
    h[12] = m*S   #3 Correct & 1 Blue
    h[13] = m*S   #3 Correct & 1 Bystander

    ai = 2
    while ai < 36:
        h[ai] = A
        ai += 4

    low = np.zeros(n)
    low[3] = 1

    valid = False
    while not valid:
        p = np.random.randint(low,high=h,size=n)

        if np.sum(p) > 0:
            valid = True
    p = p/np.sum(p)
    return p

In [48]:
def create_data_set(M, N):
    #M is the number of data points to have in this set
    #N is the number of samples to use to estimate the win percentage
    #This function will create and save a new data set file
    x_vals = []
    prob_y_vals = []
    for m in range(M):
        #Create a random A prob vector
        pA = createRandomProbVector(36)
        eA = create_events(pos_events, pA)
        # for e in eA:
            # e.printEvent()
        #Create a random B prob vector
        pB = createRandomProbVector(36)
        eB = create_events(pos_events, pB)
        #Get the win probability between them
        aLeft = 9
        bLeft = 8
        byLeft = 7

        AwinP, aw = calculateWinPercentage(eA, eB, aLeft, bLeft, byLeft, N)
        print(m, ' => ', AwinP, 'assassin wins: ', aw)
        #Create data samples (A vs B) 
        #Add to x_vals, and the corresponding wp/category into y_vals
        x_vals.append(create_vector(pA,pB))
        prob_y_vals.append(AwinP)
   
    nowTime = datetime.datetime.now ().strftime ('%Y-%m-%d-%H-%M-%S')
    filename = 'data\COLTModelData' + nowTime

    np.save(filename + 'X.npy', x_vals)
    np.save(filename + 'Y.npy', prob_y_vals)

In [49]:
#Create the datasets
num_data_sets = 1
N = 1000
M = 2000

for d in range(num_data_sets):
    create_data_set(M, N)
    print('Created dataset', d)
print('Done creating all datasets')

0  =>  0.297 assassin wins:  433
1  =>  0.37 assassin wins:  246
2  =>  0.305 assassin wins:  0
3  =>  0.636 assassin wins:  315
4  =>  0.871 assassin wins:  977
5  =>  0.398 assassin wins:  919
6  =>  0.567 assassin wins:  572
7  =>  0.571 assassin wins:  343
8  =>  0.457 assassin wins:  448
9  =>  0.966 assassin wins:  893
10  =>  0.586 assassin wins:  427
11  =>  0.412 assassin wins:  446
12  =>  0.431 assassin wins:  240
13  =>  0.749 assassin wins:  114
14  =>  0.0 assassin wins:  998
15  =>  0.627 assassin wins:  374
16  =>  0.698 assassin wins:  34
17  =>  0.903 assassin wins:  837
18  =>  0.172 assassin wins:  399
19  =>  0.585 assassin wins:  485
20  =>  0.455 assassin wins:  374
21  =>  0.001 assassin wins:  960
22  =>  0.095 assassin wins:  804
23  =>  0.759 assassin wins:  997
24  =>  0.347 assassin wins:  276
25  =>  0.364 assassin wins:  665
26  =>  0.228 assassin wins:  770
27  =>  0.445 assassin wins:  342
28  =>  0.282 assassin wins:  641
29  =>  0.398 assassin wins:  